### 3 subword algorithms help to improve your NLP model performance
- Byte Pair Encoding (BPE)
- WordPiece
- Unigram Language Model
- SentencePiece  

Subword balances vocabulary size and footprint. Extreme case is we can only use 26 token (i.e. character) to present all English word. 16k or 32k subwords are recommended vocabulary size to have a good result.

Many Asian language word cannot be separated by space. Therefore, the initial vocabulary is larger than English a lot. You may need to prepare over 10k initial word to kick start the word segmentation. From Schuster and Nakajima research, they propose to use 22k word and 11k word for Japanese and Korean respectively.  
https://medium.com/@makcedward/how-subword-helps-on-your-nlp-model-83dd1b836f46

To perform subword tokenization, BPE is slightly modified in its implementation such that the frequently occurring subword pairs are merged together instead of being replaced by another byte to enable compression. This would basically lead the rare word athazagoraphobia to be split up into more frequent subwords such as ['▁ath', 'az', 'agor', 'aphobia'].
https://towardsdatascience.com/byte-pair-encoding-the-dark-horse-of-modern-nlp-eb36c7df4f10

### Tokenizers: How machines read - 28 JANUARY 2020
Recommended read on tokenization  

- **BPE**: Just uses the frequency of occurrences to identify the best match at every iteration until it reaches the predefined vocabulary size.
- **WordPiece**: Similar to BPE and uses frequency occurrences to identify potential merges but makes the final decision based on the likelihood of the merged token
- **Unigram**: A fully probabilistic model which does not use frequency occurrences. Instead, it trains a LM using a probabilistic model, removing the token which improves the overall likelihood the least and then starting over until it reaches the final token limit.
- **SentencePiece** basically tries to bring all the subword tokenization tools and techniques under one banner. _" SentencePiece is a re-implementation of sub-word units, an effective way to alleviate the open vocabulary problems in neural machine translation. SentencePiece supports two segmentation algorithms, byte-pair-encoding (BPE) [Sennrich et al.] and unigram language model [Kudo.]. "_ (BPE and Unigram are reimplemented with improvements).
    - __All other models assume input is already tokenized__: BPE and Unigram are great models but they share one big disadvantage- they both need to have their input already tokenized. BPE needs to have the input tokenized so that every character (including word-boundary characters) are tokenized. Only then can BPE count frequencies and start to merge tokens. Usually this is done by simply doing word level tokenization but, as we discussed earlier, this is a problem with tokenization since not all languages are space segmented. Similarly, the unigram model needs to have its input tokenized before it can start discarding tokens based on their probability distribution. SentencePiece deals with this by simply taking in an input in raw text and then doing everything (which we will discuss below) needed on that input to perform subword tokenization.
    - __Encode everything as unicode ...__: SentencePiece first converts all the input into unicode characters. This means it doesn’t have to worry about different languages or characters or symbols. If it uses unicode it can just treat all input in the same way, which allows it to be language agnostic
    - __… including  the spaces__: To get around the word segmenting issues, SentencePiece simply encodes spaces as a unicode symbol. Specifically it encodes it as unicode value U+2581 (underscore ‘_’ to those of us who don’t speak unicode). This helps with the language agnostic issues and the decoding issue. Since spaces are unicode encoded then they can be easily reversed or decoded and treated (i.e learned) like a normal language character. It sounds like a simple approach and I guess it is, but the best ideas tend to seem that way in the end


https://blog.floydhub.com/tokenization-nlp/

### __Huggingface `tokenizers`__ : 
Provided Tokenizers
- CharBPETokenizer: The original BPE
- ByteLevelBPETokenizer: The byte level version of the BPE
- SentencePieceBPETokenizer: A BPE implementation compatible with the one used by SentencePiece
- BertWordPieceTokenizer: The famous Bert tokenizer, using WordPiece  
 
We designed the library so that it provides all the required blocks to create end-to-end tokenizers in an interchangeable way. In that sense, we provide
these various components: 

- **Normalizer**: Executes all the initial transformations over the initial input string. For example when you need to
lowercase some text, maybe strip it, or even apply one of the common unicode normalization process, you will add a Normalizer. 
- **PreTokenizer**: In charge of splitting the initial input string. That's the component that decides where and how to
pre-segment the origin string. The simplest example would be like we saw before, to simply split on spaces.
- **Model**: Handles all the sub-token discovery and generation, this part is trainable and really dependant
 of your input data.
- **Post-Processor**: Provides advanced construction features to be compatible with some of the Transformers-based SoTA
models. For instance, for BERT it would wrap the tokenized sentence around [CLS] and [SEP] tokens.
- **Decoder**: In charge of mapping back a tokenized input to the original string. The decoder is usually chosen according
to the `PreTokenizer` we used previously.
- **Trainer**: Provides training capabilities to each model. 

Notebook for Tokenizers: https://github.com/huggingface/transformers/blob/master/notebooks/01-training-tokenizers.ipynb  
Github Link for Python Binding: https://github.com/huggingface/tokenizers/tree/master/bindings/python

Implementation: https://github.com/huggingface/tokenizers/tree/master/bindings/python/tokenizers/implementations


## Imports

In [1]:
!pip install tokenizers

In [2]:
import os
from pathlib import Path

from tokenizers import (ByteLevelBPETokenizer,
                            CharBPETokenizer,
                            SentencePieceBPETokenizer,
                            BertWordPieceTokenizer)
from tokenizers.normalizers import BertNormalizer

## Constants

In [3]:
DATA_PATH = Path("..")

# DATA_RAW_PATH = DATA_PATH/"raw"
DATA_RAW_EXTRACTED_PATH = DATA_PATH/"raw_data_extraction"

# 1. The data from thwiki
THWIKI_FOLDER = Path("thwiki-20200601-extracted")
WIKI_FILES = list((DATA_RAW_EXTRACTED_PATH/THWIKI_FOLDER).glob("Wiki*.txt"))
list(map(print , WIKI_FILES[:5]))


# 2. The classification data from jung and ninja
CLASSIFICATION_JUNG_NINJA_FOLDER = Path("classification_dataset")
CLASSIFICATION_FILES = list((DATA_RAW_EXTRACTED_PATH/CLASSIFICATION_JUNG_NINJA_FOLDER).glob("*.txt"))
list(map(print , CLASSIFICATION_FILES[:5]))

# 3. The Data from p'Moo Crawlers
ANOTHER_WEBSITE_MOO_FOLDER = Path("another_website")
ANOTHER_WEBSITE_FILES = list((DATA_RAW_EXTRACTED_PATH/ANOTHER_WEBSITE_MOO_FOLDER).glob("*.txt"))
list(map(print , ANOTHER_WEBSITE_FILES[:5]))


# 4. Senior Project Files
SENIOR_PROJ_FOLDER = Path("data_lm")
SENIOR_PROJ_FILES = list((DATA_RAW_EXTRACTED_PATH/SENIOR_PROJ_FOLDER).glob("*.txt"))
list(map(print , SENIOR_PROJ_FILES[:5]))

# 5. Guru Crawler Files
GURU_CRAWLER_FOLDER = Path("social_listening")
GURU_CRAWLER_FILES = list((DATA_RAW_EXTRACTED_PATH/GURU_CRAWLER_FOLDER).glob("*.txt"))
list(map(print , GURU_CRAWLER_FILES[:5]))

ALL_FILES = WIKI_FILES + CLASSIFICATION_FILES + ANOTHER_WEBSITE_FILES + SENIOR_PROJ_FILES + GURU_CRAWLER_FILES
print(f"\nI have a total of {len(ALL_FILES)} files!")


# Output is in bytes - helper from Pathlib Path https://stackoverflow.com/questions/2104080/how-can-i-check-file-size-in-python
def getStat(prev_value, cur_value):
    if isinstance(prev_value, int):
        return prev_value + cur_value.stat().st_size
    return prev_value.stat().st_size + cur_value.stat().st_size

from functools import reduce
print(f"Amounts to a total of {reduce(getStat, ALL_FILES)/1e6:.2f} MB")

../raw_data_extraction/thwiki-20200601-extracted/WikiAD_3.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAE_3.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAF_0.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAF_2.txt
../raw_data_extraction/thwiki-20200601-extracted/WikiAD_1.txt
../raw_data_extraction/classification_dataset/dailynews_0.txt
../raw_data_extraction/classification_dataset/pptv36_0.txt
../raw_data_extraction/classification_dataset/prbangkok_0.txt
../raw_data_extraction/classification_dataset/siamrath_0.txt
../raw_data_extraction/classification_dataset/springnews_0.txt
../raw_data_extraction/another_website/khaosod_16.txt
../raw_data_extraction/another_website/pantip_470.txt
../raw_data_extraction/another_website/pantip_415.txt
../raw_data_extraction/another_website/naewna_2.txt
../raw_data_extraction/another_website/brighttv_5.txt
../raw_data_extraction/data_lm/Pantipdata_train.csv_351.txt
../raw_data_extraction/data_lm/Pantipdata_train.csv_81.

In [4]:
[file for file in list(map(str, ALL_FILES)) if os.path.isdir(file)]

[]

In [5]:
# import chardet
# for filename in list(map(str, ALL_FILES))[::-1]:
#     with open(filename, 'rb') as f:
#         content_bytes = f.read()
#     detected = chardet.detect(content_bytes)
#     encoding = detected['encoding']
#     print(f"{filename}: detected as {encoding}.")

In [6]:
list((DATA_RAW_EXTRACTED_PATH/CLASSIFICATION_JUNG_NINJA_FOLDER).glob("*"))

[PosixPath('../raw_data_extraction/classification_dataset/dailynews_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/pptv36_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/prbangkok_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/siamrath_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/springnews_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/naewna_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/thaipbs_0.txt'),
 PosixPath('../raw_data_extraction/classification_dataset/prachachat_0.txt')]

# Train `ByteLevelBPETokenizer` [link](https://github.com/huggingface/tokenizers/blob/master/bindings/python/tokenizers/implementations/byte_level_bpe.py#L9)

 ```python
class ByteLevelBPETokenizer(BaseTokenizer):
    """ ByteLevelBPETokenizer
    Represents a Byte-level BPE as introduced by OpenAI with their GPT-2 model
    """
    def __init__(
        self,
        vocab_file: Optional[str] = None,
        merges_file: Optional[str] = None,
        add_prefix_space: bool = False, 
        lowercase: bool = False,
        dropout: Optional[float] = None,
        unicode_normalizer: Optional[str] = None,
        continuing_subword_prefix: Optional[str] = None,
        end_of_word_suffix: Optional[str] = None,
        trim_offsets: bool = False,
```
- [`add_prefix_space`](https://github.com/huggingface/tokenizers/blob/74d812d40180032d2dbb6ca59e2e10f0257ef46b/bindings/python/tokenizers/pre_tokenizers/__init__.pyi#L26) Whether to add a space to the first word if there isn't already one. This
lets us treat `hello` exactly like `say hello`.
- [`continuing_subword_prefix`](https://github.com/huggingface/tokenizers/blob/master/bindings/python/tokenizers/implementations/byte_level_bpe.py#L33) and `end_of_word_suffix` are suffixes to add when there are words/continuing words
- [`trim_offsets`](https://github.com/huggingface/tokenizers/blob/74d812d40180032d2dbb6ca59e2e10f0257ef46b/bindings/python/tokenizers/processors/__init__.pyi#L84) This post-processor takes care of trimming the offsets. By default, the ByteLevel BPE might include whitespaces in the produced tokens. If you don't want the offsets to include these whitespaces, then this PostProcessor must be used.

In [7]:
# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer(lowercase=True,                 # adds lowercase to normalizer
                                  unicode_normalizer="nfkc",      # adds unicode normalizer nfkc to normalizer
                                  add_prefix_space=True,          # add space to the first word
                                 )
print("This is tokenizer that I will train: ", tokenizer)

This is tokenizer that I will train:  Tokenizer(vocabulary_size=0, model=ByteLevelBPE, add_prefix_space=True, lowercase=True, dropout=None, unicode_normalizer=nfkc, continuing_subword_prefix=None, end_of_word_suffix=None, trim_offsets=False)


## Special Tokens
> SentencePiece reserves vocabulary ids for special meta symbols, e.g., unknown symbol (<unk\>), BOS (<s\>), EOS (</s\>) and padding (<pad\>). Their actual ids are configured with command line flags. We can also define custom meta symbols to encode contextual information as virtual tokens. Examples include the language- indicators, <2ja> and <2de>, for multilingual models

-- From SentencePiece Paper

Bert uses the special tokens `[UNK] [CLS] [SEP] [PAD] [MASK]`

- Unknown: `[UNK]` `<unk>`
- Beginning of Sentence (BOS): `[CLS]` `<s>`
- Ending of Sentence (EOS): `[SEP]` `</s>`
- Padding: `[PAD]`  `<pad>`
- Mask: `[MASK]` `<mask>`

In [8]:
tokenizer.train(files=list(map(str, ALL_FILES)), 
                vocab_size=20000, 
                min_frequency=2,
                show_progress=True,
                special_tokens=["<s>","<pad>","</s>","<unk>","<mask>"],
               )
print("Trained vocab size: {}".format(tokenizer.get_vocab_size()))

Trained vocab size: 20000


In [19]:
!mkdir all-data-bytebpe-20000

mkdir: cannot create directory ‘all-data-bytebpe-20000’: File exists


In [20]:
# And finally save it somewhere
# ! mkdir all-data-bytebpe-30522
tokenizer.save_model("all-data-bytebpe-20000")
tokenizer.save("./all-data-bytebpe-20000.tokenizer.json", pretty=True)

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'save_model'

### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [13]:
encoded = tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

[1125, 275, 566, 278, 333, 275, 282, 16456, 327, 2565, 368, 317, 830, 340, 274, 301, 334, 301, 13155, 302, 3555, 271, 1303, 1468, 278, 6036, 271, 279, 225, 407, 302, 283, 296, 1273, 444, 271, 19117, 785, 14369, 278, 333, 275, 282, 225, 1986, 271, 12879, 301, 11970, 1474, 793, 1050]
['Ġà¸ªà¸§', 'à¸±', 'à¸ªà¸Ķ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġà¸ľà¸¡à¸Ĭ', 'à¸·à¹Ī', 'à¸Ńà¹Ħ', 'à¸Ļà¸Ĺ', 'à¹Į', 'Ġà¸ķà¸Ńà¸Ļà¸Ļ', 'à¸µà¹ī', 'à¸ģ', 'à¹ĩ', 'à¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¹Ģà¸§à¸¥à¸²à¸Ĺ', 'à¸µà¹Ī', 'à¸ľà¸¡à¸ķ', 'à¹ī', 'à¸Ńà¸ĩà¹Ħà¸Ľ', 'à¹Ĥà¸£à¸ĩà¹Ģà¸£', 'à¸µ', 'à¸¢à¸Ļà¹ģà¸¥', 'à¹ī', 'à¸§', 'Ġ', 'Ġà¸Ļ', 'à¸µà¹Ī', 'à¸Ħ', 'à¸·', 'à¸Ńà¸ģà¸²à¸£', 'à¹Ģà¸§', 'à¹ī', 'à¸Ļà¸§à¸£', 'à¸£à¸Ħ', 'à¸ªà¸Ńà¸ĩà¸Ĺ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġ', 'Ġà¸Īà¸°à¹Ħà¸Ķ', 'à¹ī', 'à¸Ńà¸Ńà¸ģà¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¸ªà¸Ńà¸ĩ', 'Ġsp', 'ac', 'es']
[' สว', 'ั', 'สด', 'ี', 'คร', 'ั', 'บ', ' ผมช', 'ื่', 'อไ', 'นท', '์', ' ตอนน', 'ี้', 'ก', '็', 'เป', '็', 'นเวลาท', 'ี่', 'ผมต', '้', 'องไป', 'โรงเร', 'ี', 'ยนแล', '้', 'ว', ' ', ' น', 'ี่', 'ค

In [14]:
encoded = tokenizer.encode(u"Hello Thisis a test in English. How is this algorithm learning?? I dunno as well.")
print(encoded.ids)
print(encoded.tokens)

[622, 15891, 6731, 837, 578, 12051, 1187, 18354, 18, 8937, 2931, 6731, 2372, 75, 638, 3830, 81, 3256, 9298, 932, 1286, 1677, 714, 1036, 9480, 3339, 731, 3169, 18]
['Ġh', 'ello', 'Ġthis', 'is', 'Ġa', 'Ġtest', 'Ġin', 'Ġenglish', '.', 'Ġhow', 'Ġis', 'Ġthis', 'Ġal', 'g', 'or', 'ith', 'm', 'Ġle', 'arn', 'ing', '??', 'Ġi', 'Ġd', 'un', 'no', 'Ġas', 'Ġw', 'ell', '.']


### Test Tokenize on Pantip Sample ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ
https://pantip.com/topic/40006922

In [15]:
text = "ใครเคยมีแฟนที่กินอาหารไม่ถูกปากกันแล้วรู้สึกเสียความสุขไปอย่างนึงบ้างมั้ยครับ  ก่อนอื่นผมต้องบอกก่อนเลยว่าคนเราจะเลือกกินอาหารแบบไหนชอบแบบไหนเป็นเรื่องของความชอบส่วนตัวนะครับทุกคนมีสิทธิในการเลือกของที่ชอบและไม่ชอบอยู่แล้ว แต่ผมรู้สึกว่าตอนนี้ผมกำลังประสบปัญหาที่ดูเหมือนจะเล็กแต่กลายเป็นว่ามันค่อนข้างใหญ่ ผมคบกับแฟนมา6ปีแล้วครับ ผมเป็นคนชอบกินอาหารญี่ปุ่นและปลาดิบแต่แฟนผมไม่กินปลาดิบเลย ผมอยากกินบุฟเฟ่เนื้อแต่แฟนผมก็ไม่กินเนื้อ เราเลยไม่ได้เข้าทานร้านบุฟเฟ่เนื้อและบุฟเฟ่อาหารญี่ปุ่นกันเพราะรู้สึกลัวแฟนผมทานไม่คุ้ม และเรื่องใหญ่เลยคือผมเป็นคนชอบทานอาหารรสจัดและรสเผ็ดมาก แต่แฟนผมทานเผ็ดไม่ได้เลยเวลาเราไปกินส้มตำกันก็จะสั่ง ส้มตำไม่ใส่พริก ต้มแซ่บไม่ใส่พริก ลาบไม่ใส่พริก ร้านกับข้าวอื่นๆก็เช่นกันแฟนผมจะไม่ชอบกินผักไม่ค่อยสั่งกับข้าวที่เป็นผักแล้วผมชอบผักบุ้งทอดกรอบ เห็ดหอมสดทอดมาก แต่ก็ไม่ได้สั่งเพราะว่าเธอไม่กินถึงเค้าจะบอกให้สั่งเลยๆก็เถอะแต่ผมก็ยังเกรงใจเธออยู่ดีอ่ะครับ ผมรู้สึกกินอาหารไม่มีความสุขเลยชีวิตผมขาดรสเผ็ดไปเหมือนจะขาดใจเหมือนมันทำให้ขาดความสุขไปอย่างนึงเลยอ่ะครับ ยิ่งถ้าเราแต่งงานกันแล้วผมก็อาจจะต้องมีปัญหาเรื่องนี้มากขึ้น พอผมเห็นคู่ที่ชอบทานอาหารเหมือนๆกันเห็นเค้ากินอาหารกันอย่างมีความสุขแล้วผมรู้สึกอิจฉามากๆเลย มีใครเคยมีปัญหาแบบผมมั้ยครับแล้วจะแก้ปัญหานี้ยังไงดีครับ"
encoded = tokenizer.encode(text)
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

[16378, 278, 7942, 302, 274, 289, 3645, 320, 266, 326, 304, 1439, 6178, 275, 1018, 271, 697, 345, 294, 330, 2377, 278, 280, 883, 305, 306, 3720, 266, 949, 330, 852, 271, 1373, 339, 1544, 275, 282, 225, 361, 266, 2353, 327, 2193, 290, 271, 9443, 266, 13457, 266, 354, 780, 5164, 296, 3251, 289, 3645, 4311, 596, 4311, 334, 301, 780, 327, 1426, 375, 10907, 266, 1088, 275, 5717, 275, 635, 305, 5843, 278, 294, 289, 637, 289, 6988, 296, 341, 1921, 302, 596, 3420, 266, 13802, 370, 324, 271, 279, 397, 266, 3525, 345, 294, 330, 438, 266, 3115, 340, 1179, 312, 423, 275, 277, 10287, 275, 3676, 302, 281, 304, 558, 296, 2028, 338, 301, 2880, 266, 2119, 301, 531, 266, 319, 275, 501, 266, 1321, 271, 10271, 266, 530, 2631, 275, 15047, 26, 297, 278, 324, 271, 1392, 275, 282, 3901, 301, 705, 4939, 289, 3645, 356, 302, 297, 450, 1216, 553, 411, 289, 2571, 266, 746, 1925, 266, 274, 289, 4679, 411, 289, 4355, 6407, 274, 289, 718, 305, 6747, 266, 457, 426, 3358, 266, 746, 1179, 301, 320, 266, 274, 289, 1955,

### Test Tokenize on Pantip Sample อาการแบบนี้คือไรกัน?
https://pantip.com/topic/40009518

In [16]:
text = "อาการแบบนี้คือไรกัน?  เขาคุยกับเรามา 5-6 เดือน เราตามจีบเขานะคะ ก็คุยกันมา ในระยะเวลาเขาบอกว่า ถ้าเราลด นน เพื่อเขาได้ เขาจะยอมเป็นแฟนเรา ตรรกะโง่มากนะคะ แต่ถามว่าทำมั้ย ทำค่ะ พอไปรู้ว่าเขาคุยกับเพื่อน เพื่อนเขาถามว่า รู้สึกยังไงกับเรา เขาตอบเพื่อนว่า เขาว่าเขาควรอยู่คนเดียว ยังไม่พร้อมจะรักใคร จนตอนนี้เราเริ่มรู้สึกว่า ทำไมเราต้องทำขนาดนั้น ถ้าเขาจะรัก รักที่เป็นตัวเราไม่ได้หรอ หลังๆเลยเริ่มสนใจเขาน้อยลง แต่ยังคุยกันเหมือนเดิม เราลองแกล้งเงียบไป ไม่ทักไปครึ่งวัน ปกติเราจะมีการมอนิ่งกันตอนเช้าค่ะ พอเราไม่ทักไป เขาทำงานเสร็จ ถึงเวลาพักของเขา เขาก็ทักมาว่า กินข้าวกัน เราก็ยิ่ง งง ก็คิดว่า เขาอาจจะชินหับการคุยกับเราทุกวันเฉยๆ นี่เลยไม่ได้สนใจในส่วนนั้น เราก็ตอบตามปกติ จนเมื่อคืนมีคนมาทักเราจีบเรา จะไปส่งเราที่บ้าน เราก็เลยเล่าให้เขาฟังว่า ให้ไลน์ไป ให้เขาไปส่งอยู่แต่ไมไ่ด้นั่งรถคันเดียวกัน เราก็ขับของเรา คนที่มาจีบเราเขาก็ขับคันของเขาแค่มาส่งเฉยๆ พอเช้ามาเขาทักมามอนิ่ง ก็ถามเราเรื่องเมื่อคืน เราทำงานที่กลับดึกมากๆไม่ได้ทักไปบอกเขาไว้ว่า ถึงบ้านแล้วนะ เงียบไปทั้งคืนเลย เขาก็ถามเรื่องเมื่อคืนว่า หนุ่มไปส่งที่บ้านเป็นไงบ้าง ถามแต่เรื่องของผู้ชายที่มาจีบเราทั้งวัน จนเราเปลี่ยนเรื่องก็ยังกลับมาถามอีกรอบ ไออาการแบบนี้คืออะไรคะ ? ไหนเขาบอกอยากอยู่คนเดียว แต่พอเรามีคนเข้ามา ทำไมเขาถึงมีอาการแบบนี้ มาถามแบบนี้ซ้ำๆ คืออะไรกัน เราไม่อยากคิดอะไรไปเอง ใครพอจะตอบได้บ้างคะ ว่า ไอแบบนี้มันคืออะไร รู้สึกอะไรอยู่"
encoded = tokenizer.encode(text)
print(encoded.ids)
print(encoded.tokens)
print(list(map(lambda x : tokenizer.decode([x]), encoded.ids)))

[16263, 340, 283, 296, 4319, 274, 275, 264, 35, 225, 7454, 305, 511, 275, 1291, 1722, 601, 17, 26, 670, 296, 325, 428, 1626, 295, 278, 1897, 3044, 361, 301, 283, 305, 511, 275, 970, 562, 5432, 816, 1940, 266, 262, 470, 271, 570, 13859, 16706, 441, 327, 1931, 680, 271, 9022, 1124, 334, 301, 3402, 459, 1555, 992, 285, 3988, 266, 396, 646, 397, 266, 1733, 266, 359, 312, 319, 339, 280, 362, 312, 354, 266, 285, 14120, 273, 345, 279, 266, 9525, 305, 511, 275, 1309, 327, 325, 441, 327, 3441, 5296, 266, 262, 409, 345, 294, 330, 1462, 275, 1846, 275, 1812, 612, 1454, 367, 327, 1715, 266, 262, 16418, 266, 816, 1830, 323, 370, 1400, 278, 425, 538, 275, 556, 266, 539, 271, 384, 1508, 275, 6341, 6322, 340, 3587, 406, 1410, 345, 294, 330, 438, 266, 262, 362, 312, 476, 1841, 271, 620, 312, 10347, 339, 264, 470, 271, 816, 3342, 275, 274, 409, 275, 551, 302, 334, 301, 466, 275, 10083, 266, 332, 271, 1393, 700, 275, 880, 16002, 406, 276, 1218, 7359, 271, 3523, 397, 266, 280, 275, 491, 305, 511, 275, 170

### If we want to use it again

The Encoding structure exposes multiple properties which are useful when working with transformers models

- normalized_str: The input string after normalization (lower-casing, unicode, stripping, etc.)
- original_str: The input string as it was provided
- tokens: The generated tokens with their string representation
- input_ids: The generated tokens with their integer representation
- attention_mask: If your input has been padded by the tokenizer, then this would be a vector of 1 for any non padded token and 0 for padded ones.
- special_token_mask: If your input contains special tokens such as [CLS], [SEP], [MASK], [PAD], then this would be a vector with 1 in places where a special token has been added.
- type_ids: If your input was made of multiple "parts" such as (question, context), then this would be a vector with for each token the segment it belongs to.
- overflowing: If your input has been truncated into multiple subparts because of a length limit (for BERT for example the sequence length is limited to 512), this will contain all the remaining overflowing parts.

In [17]:
from tokenizers import Tokenizer
tokenizer = Tokenizer.from_file("./all-data-bytebpe-30522.tokenizer.json")
encoded =  tokenizer.encode(u"สวัสดีครับ ผมชื่อไนท์ ตอนนี้ก็เป็นเวลาที่ผมต้องไปโรงเรียนแล้ว  นี่คือการเว้นวรรคสองทีครับ  จะได้ออกเป็นสอง Spaces")
print(encoded.ids)
print(encoded.tokens)

[1125, 275, 566, 278, 333, 275, 282, 16456, 327, 2565, 368, 317, 830, 340, 274, 301, 334, 301, 13155, 302, 3555, 271, 1303, 1468, 278, 6036, 271, 279, 225, 407, 302, 283, 296, 1273, 444, 271, 19117, 785, 14369, 278, 333, 275, 282, 225, 1986, 271, 12879, 301, 11970, 1474, 793, 1050]
['Ġà¸ªà¸§', 'à¸±', 'à¸ªà¸Ķ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġà¸ľà¸¡à¸Ĭ', 'à¸·à¹Ī', 'à¸Ńà¹Ħ', 'à¸Ļà¸Ĺ', 'à¹Į', 'Ġà¸ķà¸Ńà¸Ļà¸Ļ', 'à¸µà¹ī', 'à¸ģ', 'à¹ĩ', 'à¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¹Ģà¸§à¸¥à¸²à¸Ĺ', 'à¸µà¹Ī', 'à¸ľà¸¡à¸ķ', 'à¹ī', 'à¸Ńà¸ĩà¹Ħà¸Ľ', 'à¹Ĥà¸£à¸ĩà¹Ģà¸£', 'à¸µ', 'à¸¢à¸Ļà¹ģà¸¥', 'à¹ī', 'à¸§', 'Ġ', 'Ġà¸Ļ', 'à¸µà¹Ī', 'à¸Ħ', 'à¸·', 'à¸Ńà¸ģà¸²à¸£', 'à¹Ģà¸§', 'à¹ī', 'à¸Ļà¸§à¸£', 'à¸£à¸Ħ', 'à¸ªà¸Ńà¸ĩà¸Ĺ', 'à¸µ', 'à¸Ħà¸£', 'à¸±', 'à¸ļ', 'Ġ', 'Ġà¸Īà¸°à¹Ħà¸Ķ', 'à¹ī', 'à¸Ńà¸Ńà¸ģà¹Ģà¸Ľ', 'à¹ĩ', 'à¸Ļà¸ªà¸Ńà¸ĩ', 'Ġsp', 'ac', 'es']
